In [ ]:
import sqlite3
import pandas as pd

def print_table(query, p=False):
    conn = sqlite3.connect('../data/ratingRager.db')
    cursor = conn.cursor()
    cursor.execute(query)

    column_names = [description[0] for description in cursor.description]

    rows = cursor.fetchall()

    column_widths = [max(len(str(row[i])) for row in rows + [column_names]) for i in range(len(column_names))]

    header = "  ".join(f"{name:<{column_widths[i]}}" for i, name in enumerate(column_names))
    if p:
        print('\n',header)
        print("-" * len(header))

        for row in rows:
            row_str = "  ".join(f"{str(item):<{column_widths[i]}}" for i, item in enumerate(row))
            print(row_str)
        print('\n')

    return pd.read_sql_query(query, conn)

In [ ]:
QUERY = """
SELECT r.id, r.name, r.price_level
FROM restaurants as r
"""
df = print_table(QUERY)

In [ ]:
print(df['price_level'])
df.fillna({"price_level": 2}, inplace=True)


0       1.0
1       2.0
2       2.0
3       1.0
4       2.0
       ... 
3937    2.0
3938    2.0
3939    2.0
3940    2.0
3941    2.0
Name: price_level, Length: 3942, dtype: float64


In [ ]:
import numpy as np
def calculate_price_score(price_level):
    return 1 / float(price_level)

df["price_score"] = df["price_level"].apply(calculate_price_score)
print(df["price_score"])

0       1.0
1       0.5
2       0.5
3       1.0
4       0.5
       ... 
3937    0.5
3938    0.5
3939    0.5
3940    0.5
3941    0.5
Name: price_score, Length: 3942, dtype: float64


In [ ]:
# Update score table

conn = sqlite3.connect('../data/ratingRager.db')
cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(
        "UPDATE scores SET price_score = ? WHERE restaurant_id = ?;",
        (row['price_score'], row['id'])
    )

conn.commit()
conn.close()